In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

def print_map(map_):
    for row in map_:
        print(''.join(row))
        
def input_to_np_map(input_):
    rows = input_.splitlines()
    map_ = np.array([list(r) for r in rows])
    return map_

In [2]:
test = """
..F7.
.FJ|.
SJ.L7
|F--J
LJ...
""".strip()

In [3]:
with open('input.txt', 'r') as f:
    input_ = f.read().strip()

# Part 1

In [4]:
def connect_north(g, row_idx, col_idx, h, w):
    if row_idx-1 >= 0:
        g.add_edge((row_idx, col_idx), (row_idx-1, col_idx))


def connect_south(g, row_idx, col_idx, h, w):
    if row_idx+1 < h:
        g.add_edge((row_idx, col_idx), (row_idx+1, col_idx))


def connect_west(g, row_idx, col_idx, h, w):
    if col_idx-1 >= 0:
        g.add_edge((row_idx, col_idx), (row_idx, col_idx-1))


def connect_east(g, row_idx, col_idx, h, w):
     if col_idx+1 < w:
        g.add_edge((row_idx, col_idx), (row_idx, col_idx+1))   


char_to_dirs = {
    '|': (connect_north, connect_south),
    '-': (connect_east, connect_west),
    'L': (connect_north, connect_east),
    'J': (connect_north, connect_west),
    '7': (connect_south, connect_west),
    'F': (connect_south, connect_east),
}

In [5]:
def build_graph(rows):
    g = nx.DiGraph()
    h = len(rows)
    w = len(rows[0])
    for row_idx in range(h):
        for col_idx in range(w):
            g.add_node((row_idx, col_idx))          

    starting_pos = None
    row_idx = 0
    for row_idx, row in enumerate(rows):
        for col_idx, x in enumerate(row):
            pos = (row_idx, col_idx)
            # Create node in graph
            n = g.add_node(pos)
            # Create edges in graph
            dirs = char_to_dirs.get(x, None)
            if dirs is not None:
                dirs[0](g, row_idx, col_idx, h, w)
                dirs[1](g, row_idx, col_idx, h, w)
            elif x == 'S':
                starting_pos = pos
    
    return g, starting_pos

In [6]:
test = """
7-F7-
.FJ|7
SJLL7
|F--J
LJ.LJ
""".strip()

In [7]:
rows = test.splitlines()
g, starting_pos = build_graph(rows)
gr = g.reverse()
for e in gr.edges(starting_pos):
    g.add_edge(e[0], e[1])

len(list(nx.dfs_edges(g, starting_pos))) // 2 + 1

8

In [8]:
rows = input_.splitlines()
g, starting_pos = build_graph(rows)
gr = g.reverse()
for e in gr.edges(starting_pos):
    g.add_edge(e[0], e[1])

len(list(nx.dfs_edges(g, starting_pos))) // 2 + 1

6856

# Part 2

In [9]:
test = """
..........
.S------7.
.|F----7|.
.||OOOO||.
.||OOOO||.
.|L-7F-J|.
.|II||II|.
.L--JL--J.
..........
""".strip()

In [10]:
test2 = """
.F----7F7F7F7F-7....
.|F--7||||||||FJ....
.||.FJ||||||||L7....
FJL7L7LJLJ||LJ.L-7..
L--J.L7...LJS7F-7L7.
....F-J..F7FJ|L7L7L7
....L7.F7||L7|.L7L7|
.....|FJLJ|FJ|F7|.LJ
....FJL-7.||.||||...
....L---J.LJ.LJLJ...
""".strip()

In [11]:
test3 = """
FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ7F7FJ-
L---JF-JLJ.||-FJLJJ7
|F|F-JF---7F7-L7L|7|
|FFJF7L7F-JF7|JL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L
""".strip()

In [39]:
inp = test3

In [40]:
inp_map = input_to_np_map(inp)
rows = inp.splitlines()
h = len(rows)
w = len(rows[0])

g, starting_pos = build_graph(rows)
gr = g.reverse()
for e in gr.edges(starting_pos):
    g.add_edge(e[0], e[1])

loop = list(nx.dfs_edges(g, starting_pos))
loop = [l[0] for l in loop] + [loop[-1][1]]

In [41]:
print_map(inp_map)

FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ7F7FJ-
L---JF-JLJ.||-FJLJJ7
|F|F-JF---7F7-L7L|7|
|FFJF7L7F-JF7|JL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L


# Find connected components for "empty" tiles (not in loop)

In [42]:
map_ = np.full((h, w), '.', dtype=str)
for x in loop:
    map_[x[0], x[1]] = '#'
print_map(map_)

.###################
.###################
.##################.
##############.####.
##########....####..
...########...##....
..###########..#####
..##################
.....###############
.....#############..


In [43]:
m = nx.Graph()
for r in range(h):
    for c in range(w):
        if (r, c) not in loop:
            m.add_node((r, c)) 

for r in range(h):
    for c in range(w):
        if (r, c) not in loop:
            if (c+1 < w) and ((r, c+1) not in loop):
                m.add_edge((r, c), (r, c+1))
            if (r+1 < h) and ((r+1, c) not in loop):
                m.add_edge((r, c), (r+1, c))


In [44]:
components = {}
comp_size = {}
assignment = {}

for idx, comp in enumerate(nx.connected_components(m)):
    for node in comp:
        components[node] = idx
    comp_size[idx] = len(comp)
    assignment[idx] = '?'

# Go around the loop and mark tiles to left and right of path

In [45]:
diffs_to_left = {
    ((0, 1), '-'): [(-1, 0)],
    ((0, 1), 'S'): [(-1, 0)],
    ((0, 1), 'F'): [(-1, 0), (-1, -1), (-1, -2)],
    ((0, 1), 'L'): [(-1, 0)],
    ((0, -1), '-'): [(1, 0)],
    ((0, -1), 'S'): [(1, 0)],
    ((0, -1), '7'): [(1, 0)],
    ((0, -1), 'J'): [(1, 0), (1, 1), (1, 2)],
    ((1, 0), '|'): [(0, 1)],
    ((1, 0), 'S'): [(0, 1)],
    ((1, 0), '7'): [(0, 1), (-1, 1), (-2, 1)],
    ((1, 0), 'F'): [(0, 1)],
    ((-1, 0), '|'): [(0, -1)],
    ((-1, 0), 'S'): [(0, -1)],
    ((-1, 0), 'J'): [(0, -1)],
    ((-1, 0), 'L'): [(0, -1), (1, -1), (2, -1)]
}


diffs_to_right = {
    ((0, 1), '-'): [(1, 0)],
    ((0, 1), 'S'): [(1, 0)],
    ((0, 1), 'F'): [(1, 0)],
    ((0, 1), 'L'): [(1, 0), (1, -1), (1, -2)],
    ((0, -1), '-'): [(-1, 0)],
    ((0, -1), 'S'): [(-1, 0)],
    ((0, -1), '7'): [(-1, 0), (-1, 1), (-1, 2)],
    ((0, -1), 'J'): [(-1, 0)],
    ((1, 0), '|'): [(0, -1)],
    ((1, 0), 'S'): [(0, -1)],
    ((1, 0), '7'): [(0, -1)],
    ((1, 0), 'F'): [(0, -1), (-1, -1), (-2, -1)],
    ((-1, 0), '|'): [(0, 1)],
    ((-1, 0), 'S'): [(0, 1)],
    ((-1, 0), 'J'): [(0, 1), (1, 1), (2, 1)],
    ((-1, 0), 'L'): [(0, 1)]
}

In [46]:
for idx in range(1, len(loop)):
    prev = loop[idx - 1]
    curr = loop[idx]
    diff = (curr[0] - prev[0], curr[1] - prev[1])

    piece = inp_map[prev[0], prev[1]]
    dlefts = diffs_to_left[(diff, piece)]
    for dleft in dlefts:
        left = (curr[0] + dleft[0], curr[1] + dleft[1])
        if (0 <= left[0] < h) and (0 <= left[1] < w):
            if map_[left[0], left[1]] != '#':
                map_[left[0], left[1]] = 'L'
                comp_idx = components[(left[0], left[1])]
                assignment[comp_idx] = 'L'
    
    drights = diffs_to_right[(diff, piece)]
    for dright in drights:
        right = (curr[0] + dright[0], curr[1] + dright[1])
        if (0 <= right[0] < h) and (0 <= right[1] < w):
            if map_[right[0], right[1]] != '#':
                map_[right[0], right[1]] = 'R'
                comp_idx = components[(right[0], right[1])]
                assignment[comp_idx] = 'R'
    

In [47]:
print_map(map_[:,:60])

R###################
R###################
R##################R
##############L####R
##########LLLL####RR
RRR########LLL##RRRR
.R###########LL#####
.R##################
.RRRR###############
....R#############RR


# Propagate label to connected component

In [48]:
for r in range(h):
    for c in range(w):
        if (r, c) not in loop:
            comp_idx = components[(r, c)]
            map_[r, c] = assignment[comp_idx]

In [49]:
print_map(map_[:,:60])

R###################
R###################
R##################R
##############L####R
##########LLLL####RR
RRR########LLL##RRRR
RR###########LL#####
RR##################
RRRRR###############
RRRRR#############RR


# Compute total

In [50]:
tot = {'R': 0, 'L': 0, '?': 0}
for idx in assignment.keys():
    a = assignment[idx]
    tot[a] += comp_size[idx]
tot

{'R': 30, 'L': 10, '?': 0}